In [1]:
import numpy as np
import pandas as pd
import patsy

from matplotlib.pyplot import subplots
from statsmodels.datasets import get_rdataset
import sklearn.model_selection as skm
from ISLP import load_data, confusion_table
from ISLP.models import ModelSpec as MS

In [2]:
from sklearn.tree import (DecisionTreeClassifier as DTC,
                            DecisionTreeRegressor as DTR,
                            plot_tree,
                            export_text)
from sklearn.metrics import (accuracy_score, log_loss)
from sklearn.ensemble import \
    (RandomForestRegressor as RF,
    GradientBoostingRegressor as GBR)
from ISLP.bart import BART

# Fititng Classifier

In [3]:
Stock = load_data("Smarket")
Stock.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


In [4]:
formula = "Today~ Year + Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume - 1"
Y, X = patsy.dmatrices(formula, Stock)
Y = np.where(Y> 0, 1, 0)   # Predict on Zero and 1

## Splitting

In [5]:
X_train, X_test, y_train, y_test = skm.train_test_split(X, Y, test_size = 0.3, random_state = 0)

## Fitting

In [23]:
clf = DTC(criterion = 'entropy', random_state = 0, max_depth = 3)
clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

In [24]:
accuracy_score(y_train, clf.predict(X_train))

0.5634285714285714

In [25]:
accuracy_score(y_test, clf.predict(X_test))

0.5493333333333333

# Adaboosting.M1

In [16]:
import fun.ml.boosting as boost
import importlib
importlib.reload(boost)

<module 'fun.ml.boosting' from '/Users/dominicleung/projects/fun/fun/ml/boosting/__init__.py'>

In [81]:
clf = DTC(criterion = 'entropy', random_state = 3, max_depth = 3)
pred = boost.adaboost_m1(X_train, y_train, clf, M = 100)

In [82]:
accuracy_score(y_train, pred(X_train))

0.576

In [83]:
accuracy_score(y_test, pred(X_test))

0.552